In [1]:
import gzip
from collections import defaultdict
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import LinearSVC
import random
import string
import scipy.optimize
import scipy.stats
import numpy as np
def readGz(f):
  for l in gzip.open(f):
    yield eval(l)

catDict = {
  "American Restaurant": 0,
  "Bar": 1,
  "Asian Restaurant": 2,
  "European Restaurant": 3,
  "Italian Restaurant": 4,
  "Fast Food Restaurant": 5,
  "Mexican Restaurant": 6,
  "Seafood Restaurant": 7,
  "Coffee Shop": 8,
  "Sandwich Shop": 9
}

categoryCount = defaultdict(int)


review = []
users = []
reviewCat = defaultdict(list)
for l in readGz("train.json.gz"):
  review.append(l)
  users.append(l['userID'])
  if 'categoryID' in l:
    reviewCat[l['categoryID']].append(l['reviewText'])
  
    
review = [r for r in review if 'categoryID' in r]
review_train = review[:int(len(review)/2)]
review_validation = review[int(len(review)/2):]


In [47]:
import string
from nltk.corpus import stopwords
stop_list = stopwords.words("english")
wordFreq = defaultdict(float)
wordFreqCat = {}
mydiff=defaultdict(list)
for c in range(10):
  wordFreqCat[c] = defaultdict(float)

punctuation = string.punctuation
for r in review_train:
  rText = ''.join([c for c in r['reviewText'].lower() if not c in punctuation])
  for w in rText.split():
    if w in stop_list:continue
    wordFreq[w] += 1.0 # Word frequency
    wordFreqCat[r['categoryID']][w] += 1.0 # Per-category frequency
totalWords = sum(wordFreq.values())
for w in wordFreq:
  wordFreq[w] /= totalWords

for c in range(10):
  totalWordsCat = sum(wordFreqCat[c].values())
  for w in wordFreqCat[c]:
    wordFreqCat[c][w] /= totalWordsCat

for c in range(10):
  diffs = []
  for w in wordFreq:
    diffs.append((wordFreqCat[c][w] - wordFreq[w], w))
  diffs.sort()
  diffs.reverse()
  topten = diffs[:250]
  for r in topten: mydiff[c].append(r[1])

most_common = [(wordFreq[w], w) for w in wordFreq]
most_common.sort()
most_common.reverse()
most_common_words = [w[1] for w in most_common[:500]]


In [48]:

commonWords = most_common_words
pos = dict(zip(commonWords, range(len(commonWords))))
word_set = set(commonWords)

def jacard_review(r1,r2):
  raw1 = ''.join([x for x in r1.lower() if not x in string.punctuation])
  raw2 = ''.join([x for x in r2.lower() if not x in string.punctuation])
  set_1 = set([w for w in raw1.split()])
  set_2 = set([w for w in raw2.split()])
  n = float(len(set_1.intersection(set_2)))
  denominator = float(len(set_1) + len(set_2) - n)
  if denominator == 0.0: return 0.0
  return n /denominator 

def jacard_r(user, review):
  value = 0.0
  if user not in I_u or business not in U_i:
    return value
  for i in I_u[user]:
    pearson_val = pearson_business(i['businessID'], business)
    if  pearson_val > 0.5:
        value = pearson_val
        return value  
  return value

def feature(r):
  feat = [0] * len(commonWords)
  raw = ''.join([x for x in r['reviewText'].lower() if not x in string.punctuation])
  for word in raw.split():
    if word in word_set:
      feat[pos[word]] =1 

  for i in range(10): 
    sub_feat = [0] * 250
    common0= set(mydiff[i])
    pos0 = dict(zip(common0, range(250)))
    raw = ''.join([x for x in r['reviewText'].lower() if not x in string.punctuation])
    for word in raw.split():
      if word in common0:
        sub_feat[pos0[word]] =1 
        
    feat+=sub_feat
  
  return feat

def userfeature(r):
  u = r['userID']
  total = user_activity_allcat[u]
  feat=[0]
  feat.append(0.0 if total == 0 else user_activity_cat0[u]*1.0/total)
  feat.append(0.0 if total == 0 else user_activity_cat1[u]*1.0/total)
  feat.append(0.0 if total == 0 else user_activity_cat2[u]*1.0/total)
  feat.append(0.0 if total == 0 else user_activity_cat3[u]*1.0/total)
  feat.append(0.0 if total == 0 else user_activity_cat4[u]*1.0/total)
  feat.append(0.0 if total == 0 else user_activity_cat5[u]*1.0/total)  
  feat.append(0.0 if total == 0 else user_activity_cat6[u]*1.0/total)  
  feat.append(0.0 if total == 0 else user_activity_cat7[u]*1.0/total)  
  feat.append(0.0 if total == 0 else user_activity_cat8[u]*1.0/total)  
  feat.append(0.0 if total == 0 else user_activity_cat9[u]*1.0/total)   
  
  #feat.append(abs(r['rating']-r0))
  #feat.append(abs(r['rating']-r1))
  #feat.append(abs(r['rating']-r2))  
  #feat.append(abs(r['rating']-r3))
  #feat.append(abs(r['rating']-r4))  
  #feat.append(abs(r['rating']-r5))  
  #feat.append(abs(r['rating']-r6))  
  #feat.append(abs(r['rating']-r7))  
  #feat.append(abs(r['rating']-r8))  
  #feat.append(abs(r['rating']-r9)) 
  return feat




In [44]:
x_train = [feature(r) for r in review_train]
y_train = [r['categoryID'] for r in review_train]
           
x_validation = [feature(r) for r in review_validation]
y_validation = [r['categoryID'] for r in review_validation]
clf = OneVsRestClassifier(LinearSVC(C=0.1,random_state=0))
clf.fit(x_train, y_train)
predict = clf.predict(x_validation)
acc = [(x==y) for (x,y) in zip(predict, y_validation)] 
acc = sum(acc)*1.0/len(acc)  

print(acc)

0.630947632344


In [49]:
x_train = [feature(r) for r in review]
y_train = [r['categoryID'] for r in review]
clf = OneVsRestClassifier(LinearSVC(C=0.1,random_state=0))
clf.fit(x_train, y_train)
#x_train = [feature(r) for r in review_train]
#y_train = [r['categoryID'] for r in review]
#clf = OneVsRestClassifier(LinearSVC(C=1,random_state=0))
#clf.fit(x_train, y_train)
predictions = open("predictions_Category.txt", 'w')
predictions.write("userID-reviewHash,category\n")

for l in readGz("test_Category.json.gz"):
  feat = feature(l)
  prediction = int(clf.predict([feat])[0])
  _ = predictions.write(l['userID'] + '-' + l['reviewHash'] + "," + str(prediction) + "\n")

predictions.close()

print("done")






done


In [184]:
  if l['categoryID'] == 0:
     user_activity_cat0[r['userID']]+=1
  elif l['categoryID'] == 1:
     user_activity_cat1[l['userID']]+=1
  elif l['categoryID'] == 2:
     user_activity_cat2[l['userID']]+=1
  elif l['categoryID'] == 3:
     user_activity_cat3[l['userID']]+=1
  elif l['categoryID'] == 4:
     user_activity_cat4[l['userID']]+=1
  elif l['categoryID'] == 5:
     user_activity_cat5[l['userID']]+=1
  elif l['categoryID'] == 6:
     user_activity_cat6[l['userID']]+=1
  elif l['categoryID'] == 7:
     user_activity_cat7[l['userID']]+=1
  elif l['categoryID'] == 8:
     user_activity_cat8[l['userID']]+=1
  elif l['categoryID'] == 9:
     user_activity_cat9[l['userID']]+=1
        

feat.append(l0)
  feat.append(l1)
  feat.append(l2)
  feat.append(l3)
  feat.append(l4)
  feat.append(l5)  
  feat.append(l6)  
  feat.append(l7)  
  feat.append(l8)  
  feat.append(l9)  
                
        
if word in set0:
      l0=1
    if word in set1:
      l1=1
    if word in set2:
      l2=1
    if word in set3:
      l3=1
    if word in set4:
      l4=1
    if word in set5:
      l5=1
    if word in set6:
      l6=1
    if word in set7:
      l7=1
    if word in set8:
      l8=1
    if word in set9:
      l9=1

NameError: name 'l0' is not defined